In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [3]:
print(tf.__version__)

2.0.0


In [4]:
train = pd.read_csv('train.csv')

In [5]:
X,Y = (train.drop('label',axis=1),train['label'])
X = X.values.reshape(42000,28,28,1)
X_train, X_test = (X[:38000],X[38000:])
Y_train, Y_test = (Y[:38000],Y[38000:])

In [6]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(  rotation_range=10,
                                                            width_shift_range=0.1,
                                                            height_shift_range=0.1,
                                                            zoom_range=0.1,
                                                            rescale=1./255)
datagen.fit(X_train)
datagen_valid = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
datagen_valid.fit(X_test)

In [7]:
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding = 'same',input_shape=(28,28,1)),
                                   tf.keras.layers.BatchNormalization(),
                                   
                                   tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding = 'same'),
                                   tf.keras.layers.BatchNormalization(),
                                   tf.keras.layers.MaxPooling2D(2,2), 
                                   
                                   tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding = 'same'),
                                   tf.keras.layers.BatchNormalization(),
                                    
                                   tf.keras.layers.Conv2D(192,(3,3),activation='relu',padding = 'same'),
                                   tf.keras.layers.BatchNormalization(),
                                   tf.keras.layers.MaxPooling2D(2,2), 
                                        
                                  tf.keras.layers.Flatten(),
                                  tf.keras.layers.Dense(512,activation='relu'),
                                  tf.keras.layers.BatchNormalization(),
                                  tf.keras.layers.Dense(256,activation='relu'),
                                  tf.keras.layers.BatchNormalization(),
                                  tf.keras.layers.Dense(10,activation='softmax')])

model.compile(optimizer='RMSProp',loss='sparse_categorical_crossentropy',metrics=['acc'])

In [8]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)
checkpoint = tf.keras.callbacks.ModelCheckpoint('mnist_weights.h5', monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')

In [9]:
model.load_weights('mnist_weights.h5')

In [25]:
history1 = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=128),epochs=15,validation_data=datagen_valid.flow(X_test,Y_test,batch_size=16),callbacks=[reduce_lr,checkpoint])

In [30]:
#model.save_weights('mnist_weights.h5')
#model.save(r'C:\Users\user\Desktop\Kaggle_2.0\myfirstmodel.h5')

In [175]:
test = pd.read_csv('test.csv')
test = test.values.reshape(test.shape[0],28,28,1)
test = test/255
prediction = model.predict(test)
pred_classes = prediction.argmax(axis=-1)

In [27]:
pred_classes.shape

In [176]:
pred_classes = pd.DataFrame({'ImageId': range(1,28001),'Label':pred_classes})
pred_classes.to_csv(r'C:\Users\user\Desktop\Kaggle_2.0\submission.csv',index=False)

In [26]:
output

In [17]:
output = tf.keras.layers.Dense(512, activation='relu')(model.layers[-6].output)
output = tf.keras.layers.BatchNormalization()(output)
output = tf.keras.layers.Dense(512, activation='relu')(output)
output = tf.keras.layers.BatchNormalization()(output)
output = tf.keras.layers.Dense(256, activation='relu')(output)
output = tf.keras.layers.BatchNormalization()(output)
output = tf.keras.layers.Dense(10, activation='softmax')(output)

In [18]:
inputs = model.layers[0].input

In [24]:
new_model = tf.keras.Model(inputs,output)
new_model.summary()

In [21]:
new_model.compile(optimizer='RMSProp',loss='sparse_categorical_crossentropy',metrics=['acc'])

In [23]:
history2 = new_model.fit_generator(datagen.flow(X_train, Y_train, batch_size=128),epochs=30,validation_data=datagen_valid.flow(X_test,Y_test,batch_size=16),callbacks=[reduce_lr])

In [151]:
#new_model.save_weights('model2_weights')